# Sensor Projection

In this tutorial, we will demonstrate how to define a **sensor geometry** and project its trace onto the **Earth**.

## Setup

Let's install the necessary libraries:

In [1]:
import sys

! {sys.executable} -m pip install --quiet LibraryCorePy
! {sys.executable} -m pip install --quiet LibraryIOPy
! {sys.executable} -m pip install --quiet LibraryMathematicsPy
! {sys.executable} -m pip install --quiet LibraryPhysicsPy

And import the necessary objects:

In [2]:
import numpy as np
import pandas as pd

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

In [3]:
import Library.Core as Core
import Library.IO as IO
import Library.Mathematics as Mathematics
import Library.Physics as Physics

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

to-Python converter for library::physics::time::DateTime already registered; second conversion method ignored.

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

to-Python converter for library::math::obj::IntervalBase::Type already registered; second conversion method ignored.



Then, we define some useful shortcuts:

In [4]:
Object2 = Mathematics.Geometry.D2.Object
Point2 = Mathematics.Geometry.D2.Objects.Point
Polygon2 = Mathematics.Geometry.D2.Objects.Polygon
Point3 = Mathematics.Geometry.D3.Objects.Point
Polygon3 = Mathematics.Geometry.D3.Objects.Polygon
Ellipsoid = Mathematics.Geometry.D3.Objects.Ellipsoid
Pyramid = Mathematics.Geometry.D3.Objects.Pyramid

Length = Physics.Units.Length
Angle = Physics.Units.Angle
Scale = Physics.Time.Scale
Instant = Physics.Time.Instant
Duration = Physics.Time.Duration
Interval = Physics.Time.Interval
DateTime = Physics.Time.DateTime
LLA = Physics.Coordinate.Spherical.LLA
Position = Physics.Coordinate.Position
Frame = Physics.Coordinate.Frame
Environment = Physics.Environment
Geometry = Physics.Environment.Object.Geometry
Earth = Physics.Environment.Objects.CelestialBodies.Earth

---

## Scene

We first set up a simple scene, with the Earth only (the default `Environment` will suffice here):

In [10]:
environment = Environment.Default() ;

RuntimeError: Error using cURL to fetch file at URL [https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls]: [Out of memory].

Then, we access the `Earth` object (managed by the `Environment`):

In [ ]:
earth = environment.accessObjectWithName("Earth")

Once the `Earth` has been obtained, we can also get its geometry (an `Ellipsoid`, defined in `ITRF` in this case):

In [ ]:
earth_geometry = earth.getGeometryIn(Frame.ITRF())

## Sensor

Let's define a pyramidal geometry:

In [ ]:
apex = Point3(7000e3, 0.0, 0.0)
base = Polygon3(Polygon2([Point2(-1.0, -1.0), Point2(+1.0, -1.0), Point2(+1.0, +1.0), Point2(-1.0, +1.0)]), apex - np.array((0.8, 0.0, 0.0)), np.array((0.0, 1.0, 0.0)), np.array((0.0, 0.0, 1.0)))

pyramid = Pyramid(base, apex)

That we express in `ITRF`:

In [ ]:
sensor_geometry = Geometry(pyramid, Frame.ITRF())

## Intersection

Now that we have both Earth and sensor geometries clearly defined, we can compute their intersection:

In [ ]:
intersection_ITRF = sensor_geometry.intersectionWith(earth_geometry)

And convert this 3D intersection into a 2D set of geodetic points:

In [ ]:
intersection_points = [Point2(lla.getLongitude().inDegrees(), lla.getLatitude().inDegrees()) for lla in [LLA.Cartesian(point_ITRF.asVector(), Earth.EquatorialRadius, Earth.Flattening) for point_ITRF in intersection_ITRF.accessComposite().accessObjectAt(0).asLineString()]]

We further convert this set into a Pandas Dataframe, as a very convenient way for storing / managing data in Python:

In [ ]:
intersection_df = pd.DataFrame([[float(intersection_point.x()), float(intersection_point.y())] for intersection_point in intersection_points], columns=['Longitude', 'Latitude']) ;

In [ ]:
intersection_df.head()

Now, we're ready to visualize the intersection on a map!

In [ ]:
data = []

data.append(
    dict(
        type = 'scattergeo',
        lon = intersection_df['Longitude'],
        lat = intersection_df['Latitude'],
        mode = 'lines',
        line = dict(
            width = 1,
            color = 'red',
        )
    )
)
    
layout = dict(
        title = None,
        showlegend = False, 
        geo = dict(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
        ),
    )
    
fig = dict(data=data, layout=layout)
iplot(fig)

It is also possible to obtain the `WKT` representation of the intersection polygon:

In [ ]:
Polygon2(intersection_points).toString(Object2.Format.WKT)